<a href="https://colab.research.google.com/github/venk-meg/STRIKE/blob/main/Copy_of_Parsing_CSV_M%26R_trials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STRIKE Gesture Data Processing

This notebook processes gesture recognition data for multiple gestures captured during trials by Megha and Ria. The raw data is split into individual repetitions, saved as CSVs, and zipped for downstream analysis or model training.


In [ ]:
# Clone repo if not already cloned
!git clone https://github.com/venk-meg/STRIKE.git
%cd STRIKE

# Imports
import pandas as pd
import os
import glob
from google.colab import files
import zipfile

# Constants
GESTURE_NAMES = [
    "deleterecording", "listen", "makefist",
    "nextrecording", "releasefist", "selectend", "slice"
]
NUM_REPETITIONS = 10
ZIP_FOLDER = "parsed_gesture_data"
os.makedirs(ZIP_FOLDER, exist_ok=True)

def process_gesture_data(input_template, suffix=""):
    """
    Splits gesture CSVs into individual repetitions and zips them.
    """
    for gesture in GESTURE_NAMES:
        input_file = input_template.format(gesture=gesture)
        output_dir = f"data_ml/{gesture}/split/"
        os.makedirs(output_dir, exist_ok=True)

        # Load CSV
        df = pd.read_csv(input_file)
        timesteps = len(df) // NUM_REPETITIONS

        # Split and save each repetition
        for i in range(NUM_REPETITIONS):
            start = i * timesteps
            end = len(df) if i == NUM_REPETITIONS - 1 else start + timesteps
            df_rep = df.iloc[start:end]

            filename = f"{gesture}_{i+1:02d}{suffix}.csv"
            df_rep.to_csv(os.path.join(output_dir, filename), index=False)

        # Zip the split files
        zip_path = os.path.join(ZIP_FOLDER, f"{gesture}.zip")
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            for file in glob.glob(f"{output_dir}/*.csv"):
                zipf.write(file, arcname=os.path.basename(file))

# Process Megha's and Ria's gesture trials
process_gesture_data("data_ml/{gesture}/trial_{gesture}_10x_25hz_M.csv")
process_gesture_data("data_ml/{gesture}/trial_{gesture}_10x_25hz_R.csv", suffix="_R")

# Download all zip files
for zip_file in glob.glob(f"{ZIP_FOLDER}/*.zip"):
    files.download(zip_file)
